# Stock Adjustment Barcode - Main Workflow Test
## Testing Backdated Refresh, Lot Management, and Cost Valuation

This notebook provides comprehensive testing for:
1. Backdated stock refresh functionality
2. Lot selection based on FIFO with difference calculations
3. Total cost difference and valuation tracking
4. Complete workflow from scanning to posting

## 1. Environment Setup and Imports

In [ ]:
# Import required modules
import sys
import os
from datetime import datetime, timedelta
from decimal import Decimal
import pandas as pd
import json

# Add Odoo to path (adjust path as needed)
sys.path.append('/Users/jamshid/PycharmProjects/Siafa/odoo16e_simc')

# Import Odoo
import odoo
from odoo import api, fields, models, SUPERUSER_ID
from odoo.tests import TransactionCase
from odoo.exceptions import UserError, ValidationError

# Initialize Odoo (adjust config path)
odoo.tools.config.parse_config(['--config=/path/to/odoo.conf'])

# Connect to database
db_name = 'test_db'  # Change to your database
uid = SUPERUSER_ID
context = {}

# Get registry and cursor
registry = odoo.registry(db_name)
with registry.cursor() as cr:
    env = api.Environment(cr, uid, context)
    print(f"Connected to database: {db_name}")
    print(f"Company: {env.company.name}")

## 2. Test Configuration and Setup

In [ ]:
# Test configuration parameters - MODIFY THESE WITH YOUR ACTUAL DATA
TEST_CONFIG = {
    'company_id': 1,  # Your company ID
    'location_id': 8,  # Your stock location ID (e.g., WH/Stock)
    'products': [
        {
            'id': 10,  # Product ID
            'name': 'Product A',
            'barcode': 'PROD-A-001',
            'standard_price': 100.0,
            'tracking': 'lot',  # lot/serial/none
        },
        {
            'id': 11,
            'name': 'Product B',
            'barcode': 'PROD-B-001',
            'standard_price': 75.0,
            'tracking': 'lot',
        }
    ],
    'lots': [
        {'name': 'LOT-A-001', 'product_id': 10},
        {'name': 'LOT-A-002', 'product_id': 10},
        {'name': 'LOT-A-003', 'product_id': 10},
        {'name': 'LOT-B-001', 'product_id': 11},
        {'name': 'LOT-B-002', 'product_id': 11},
    ],
    'backdate_days': 7,  # How many days back for backdated testing
    'scan_quantities': {  # Barcode: quantity to scan
        'PROD-A-001': 25,
        'PROD-B-001': 15,
    }
}

print("Test configuration loaded")
print(f"Testing with {len(TEST_CONFIG['products'])} products and {len(TEST_CONFIG['lots'])} lots")

## 3. Helper Functions

In [ ]:
def create_test_data(env):
    """Create or get test products and lots"""
    products = {}
    lots = {}
    
    # Get or create products
    for prod_config in TEST_CONFIG['products']:
        product = env['product.product'].browse(prod_config['id'])
        if not product.exists():
            product = env['product.product'].create({
                'name': prod_config['name'],
                'type': 'product',
                'barcode': prod_config['barcode'],
                'standard_price': prod_config['standard_price'],
                'tracking': prod_config['tracking'],
            })
        products[prod_config['barcode']] = product
    
    # Get or create lots
    for lot_config in TEST_CONFIG['lots']:
        lot = env['stock.lot'].search([
            ('name', '=', lot_config['name']),
            ('product_id', '=', lot_config['product_id'])
        ], limit=1)
        if not lot:
            lot = env['stock.lot'].create({
                'name': lot_config['name'],
                'product_id': lot_config['product_id'],
                'company_id': TEST_CONFIG['company_id'],
            })
        lots[lot_config['name']] = lot
    
    return products, lots


def create_initial_stock(env, location_id, products, lots):
    """Create initial stock with specific quantities and dates"""
    stock_data = []
    
    # Define initial stock levels with different in_dates for FIFO
    initial_stock = [
        # Product A lots
        {'product': 'PROD-A-001', 'lot': 'LOT-A-001', 'qty': 10, 'days_ago': 10, 'cost': 95.0},
        {'product': 'PROD-A-001', 'lot': 'LOT-A-002', 'qty': 15, 'days_ago': 7, 'cost': 98.0},
        {'product': 'PROD-A-001', 'lot': 'LOT-A-003', 'qty': 5, 'days_ago': 3, 'cost': 102.0},
        # Product B lots
        {'product': 'PROD-B-001', 'lot': 'LOT-B-001', 'qty': 20, 'days_ago': 8, 'cost': 73.0},
        {'product': 'PROD-B-001', 'lot': 'LOT-B-002', 'qty': 10, 'days_ago': 5, 'cost': 77.0},
    ]
    
    location = env['stock.location'].browse(location_id)
    
    for stock in initial_stock:
        product = products[stock['product']]
        lot = lots[stock['lot']]
        in_date = datetime.now() - timedelta(days=stock['days_ago'])
        
        # Create quant
        quant = env['stock.quant'].create({
            'product_id': product.id,
            'location_id': location_id,
            'lot_id': lot.id,
            'quantity': stock['qty'],
            'in_date': in_date,
            'company_id': TEST_CONFIG['company_id'],
        })
        
        stock_data.append({
            'product': product.name,
            'lot': lot.name,
            'quantity': stock['qty'],
            'in_date': in_date.strftime('%Y-%m-%d'),
            'cost': stock['cost'],
            'total_value': stock['qty'] * stock['cost']
        })
    
    return pd.DataFrame(stock_data)


def calculate_stock_value(env, location_id, product_ids=None):
    """Calculate total stock value for products in location"""
    domain = [('location_id', '=', location_id)]
    if product_ids:
        domain.append(('product_id', 'in', product_ids))
    
    quants = env['stock.quant'].search(domain)
    
    valuation_data = []
    total_value = 0
    
    for quant in quants:
        product = quant.product_id
        # Get product cost
        if product.cost_method == 'average':
            unit_cost = product.standard_price
        else:
            # For FIFO/LIFO, get from valuation layers
            unit_cost = product.standard_price
        
        value = quant.quantity * unit_cost
        total_value += value
        
        valuation_data.append({
            'product': product.name,
            'lot': quant.lot_id.name if quant.lot_id else 'N/A',
            'quantity': quant.quantity,
            'unit_cost': unit_cost,
            'total_value': value
        })
    
    return pd.DataFrame(valuation_data), total_value


def display_stock_summary(env, location_id):
    """Display current stock summary"""
    df, total = calculate_stock_value(env, location_id)
    print("\n=== Current Stock Summary ===")
    print(df.to_string(index=False))
    print(f"\nTotal Stock Value: ${total:,.2f}")
    return df, total

## 4. Initialize Test Environment

In [ ]:
with registry.cursor() as cr:
    env = api.Environment(cr, uid, context)
    
    # Clear existing test data (optional)
    print("Clearing existing test adjustments...")
    existing_adjustments = env['stock.adjustment.barcode'].search([
        ('location_id', '=', TEST_CONFIG['location_id']),
        ('state', 'in', ['draft', 'to_approve', 'approved'])
    ])
    if existing_adjustments:
        existing_adjustments.unlink()
        print(f"Removed {len(existing_adjustments)} existing adjustments")
    
    # Create test data
    products, lots = create_test_data(env)
    print(f"Created/Found {len(products)} products and {len(lots)} lots")
    
    # Create initial stock
    print("\nCreating initial stock...")
    initial_stock_df = create_initial_stock(env, TEST_CONFIG['location_id'], products, lots)
    print("\nInitial Stock Created:")
    print(initial_stock_df.to_string(index=False))
    
    # Display initial valuation
    initial_valuation_df, initial_total_value = display_stock_summary(env, TEST_CONFIG['location_id'])
    
    cr.commit()
    print("\n✅ Test environment initialized successfully")

## 5. Create Stock Adjustment and Scan Products

In [ ]:
with registry.cursor() as cr:
    env = api.Environment(cr, uid, context)
    
    # Create stock adjustment
    adjustment = env['stock.adjustment.barcode'].create({
        'location_id': TEST_CONFIG['location_id'],
        'company_id': TEST_CONFIG['company_id'],
        'fetch_type': 'current',
    })
    
    print(f"Created Stock Adjustment: {adjustment.name}")
    print(f"State: {adjustment.state}")
    print(f"Location: {adjustment.location_id.complete_name}")
    print(f"Inventory Date: {adjustment.inventory_date}")
    
    # Simulate barcode scanning
    scan_results = []
    for barcode, quantity in TEST_CONFIG['scan_quantities'].items():
        print(f"\nScanning {barcode} - {quantity} times...")
        for i in range(quantity):
            try:
                adjustment.on_barcode_scanned(barcode)
                if i == 0 or i == quantity - 1:
                    print(f"  Scan {i+1}/{quantity} successful")
            except Exception as e:
                print(f"  Error scanning: {e}")
                break
        
        scan_results.append({
            'barcode': barcode,
            'scanned_qty': quantity,
            'product': products[barcode].name
        })
    
    # Display scan summary
    print("\n=== Scan Summary ===")
    scan_df = pd.DataFrame(scan_results)
    print(scan_df.to_string(index=False))
    
    # Display line info
    print(f"\nTotal Line Info Records: {len(adjustment.inv_adjustment_line_info_ids)}")
    print(f"Total Adjustment Lines: {len(adjustment.inv_adjustment_line_ids)}")
    
    adjustment_id = adjustment.id
    cr.commit()
    print(f"\n✅ Scanning completed. Adjustment ID: {adjustment_id}")

## 6. Confirm Adjustment and Check Initial Calculations

In [ ]:
with registry.cursor() as cr:
    env = api.Environment(cr, uid, context)
    
    adjustment = env['stock.adjustment.barcode'].browse(adjustment_id)
    
    # Confirm adjustment
    print("Confirming adjustment...")
    adjustment.action_confirm()
    
    print(f"State after confirmation: {adjustment.state}")
    print(f"Adjustment Lines: {len(adjustment.inv_adjustment_line_ids)}")
    
    # Display adjustment lines with calculations
    line_data = []
    for line in adjustment.inv_adjustment_line_ids:
        line_data.append({
            'Product': line.product_id.name,
            'On Hand': line.on_hand_qty,
            'Scanned': line.total_scanned_qty,
            'Difference': line.difference_qty,
            'Has Error': 'Yes' if line.is_editable else 'No'
        })
    
    print("\n=== Adjustment Lines (Before Refresh) ===")
    lines_df = pd.DataFrame(line_data)
    print(lines_df.to_string(index=False))
    
    # Display lot details
    print("\n=== Lot Line Details ===")
    for line in adjustment.inv_adjustment_line_ids:
        if line.adjustment_line_lot_ids:
            print(f"\nProduct: {line.product_id.name}")
            lot_data = []
            for lot_line in line.adjustment_line_lot_ids:
                lot_data.append({
                    'Lot': lot_line.lot_id.name,
                    'Current Qty': lot_line.current_qty,
                    'New Qty': lot_line.new_qty,
                    'Difference': lot_line.difference_qty
                })
            lot_df = pd.DataFrame(lot_data)
            print(lot_df.to_string(index=False))
    
    cr.commit()
    print("\n✅ Adjustment confirmed")

## 7. Test Backdated Refresh Stock

In [ ]:
with registry.cursor() as cr:
    env = api.Environment(cr, uid, context)
    
    adjustment = env['stock.adjustment.barcode'].browse(adjustment_id)
    
    # Calculate backdate
    backdate = datetime.now() - timedelta(days=TEST_CONFIG['backdate_days'])
    
    print(f"Testing backdated refresh to: {backdate.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"Current inventory date: {adjustment.inventory_date}")
    
    # Create and apply refresh wizard
    wizard = env['stock.refresh.stock.wizard'].with_context(
        active_model='stock.adjustment.barcode',
        active_id=adjustment.id
    ).create({
        'fetch_type': 'backdated',
        'backdate': backdate
    })
    
    print("\nApplying backdated refresh...")
    wizard.action_apply()
    
    # Reload adjustment
    adjustment = env['stock.adjustment.barcode'].browse(adjustment_id)
    
    print(f"\nInventory date after refresh: {adjustment.inventory_date}")
    print(f"Fetch type: {adjustment.fetch_type}")
    
    # Display updated calculations
    line_data = []
    for line in adjustment.inv_adjustment_line_ids:
        line_data.append({
            'Product': line.product_id.name,
            'On Hand (Backdated)': line.on_hand_qty,
            'Scanned': line.total_scanned_qty,
            'Difference': line.difference_qty,
            'Unit Price': line.unit_price if line.unit_price else 0
        })
    
    print("\n=== Adjustment Lines (After Backdated Refresh) ===")
    lines_df = pd.DataFrame(line_data)
    print(lines_df.to_string(index=False))
    
    # Recalculate lot lines after refresh
    print("\n=== Updated Lot Line Details (FIFO Based) ===")
    for line in adjustment.inv_adjustment_line_ids:
        if line.adjustment_line_lot_ids:
            print(f"\nProduct: {line.product_id.name}")
            print(f"Total Difference: {line.difference_qty}")
            lot_data = []
            total_diff = 0
            for lot_line in line.adjustment_line_lot_ids.sorted('lot_id'):
                lot_data.append({
                    'Lot': lot_line.lot_id.name,
                    'Current Qty': lot_line.current_qty,
                    'New Qty': lot_line.new_qty,
                    'Difference': lot_line.difference_qty
                })
                total_diff += lot_line.difference_qty
            lot_df = pd.DataFrame(lot_data)
            print(lot_df.to_string(index=False))
            print(f"Total lot differences: {total_diff}")
    
    cr.commit()
    print("\n✅ Backdated refresh completed")

## 8. Approve and Post Adjustment

In [ ]:
with registry.cursor() as cr:
    env = api.Environment(cr, uid, context)
    
    adjustment = env['stock.adjustment.barcode'].browse(adjustment_id)
    
    # Approve adjustment
    print("Approving adjustment...")
    adjustment.action_approved()
    print(f"State: {adjustment.state}")
    print(f"Approved by: {adjustment.approved_by.name if adjustment.approved_by else 'System'}")
    
    # Calculate expected valuation impact
    print("\n=== Expected Valuation Impact ===")
    total_cost_impact = 0
    valuation_data = []
    
    for line in adjustment.inv_adjustment_line_ids:
        product = line.product_id
        # Use standard price or get from valuation
        unit_cost = product.standard_price
        cost_impact = line.difference_qty * unit_cost
        total_cost_impact += cost_impact
        
        valuation_data.append({
            'Product': product.name,
            'Difference Qty': line.difference_qty,
            'Unit Cost': unit_cost,
            'Cost Impact': cost_impact
        })
    
    valuation_df = pd.DataFrame(valuation_data)
    print(valuation_df.to_string(index=False))
    print(f"\nTotal Expected Cost Impact: ${total_cost_impact:,.2f}")
    
    # Post adjustment
    print("\n" + "="*50)
    print("Posting adjustment...")
    adjustment.action_done()
    
    print(f"State: {adjustment.state}")
    print(f"Posted Date: {adjustment.posted_date}")
    
    # Check stock moves created
    stock_moves = adjustment.inv_adjustment_line_ids.mapped('stock_move_ids')
    print(f"\nStock Moves Created: {len(stock_moves)}")
    
    # Display move details
    print("\n=== Stock Moves ===")
    move_data = []
    for move in stock_moves:
        move_data.append({
            'Product': move.product_id.name,
            'From': move.location_id.name,
            'To': move.location_dest_id.name,
            'Quantity': move.product_uom_qty,
            'State': move.state
        })
    
    if move_data:
        moves_df = pd.DataFrame(move_data)
        print(moves_df.to_string(index=False))
    
    # Update unit prices in lines
    print("\n=== Final Unit Prices Stored ===")
    for line in adjustment.inv_adjustment_line_ids:
        if line.difference_qty != 0:
            print(f"{line.product_id.name}: ${line.unit_price:.2f}")
    
    cr.commit()
    print("\n✅ Adjustment posted successfully")

## 9. Analyze Final Valuation and Cost Impact

In [ ]:
with registry.cursor() as cr:
    env = api.Environment(cr, uid, context)
    
    adjustment = env['stock.adjustment.barcode'].browse(adjustment_id)
    
    print("=" * 60)
    print("FINAL VALUATION ANALYSIS")
    print("=" * 60)
    
    # Get final stock valuation
    final_valuation_df, final_total_value = display_stock_summary(env, TEST_CONFIG['location_id'])
    
    # Calculate actual cost impact
    actual_cost_impact = final_total_value - initial_total_value
    
    print(f"\n📊 Valuation Summary:")
    print(f"  Initial Total Value: ${initial_total_value:,.2f}")
    print(f"  Final Total Value: ${final_total_value:,.2f}")
    print(f"  Actual Cost Impact: ${actual_cost_impact:,.2f}")
    print(f"  Expected Cost Impact: ${total_cost_impact:,.2f}")
    print(f"  Variance: ${abs(actual_cost_impact - total_cost_impact):,.2f}")
    
    # Analyze by product
    print("\n=== Cost Impact by Product ===")
    product_impact = []
    for line in adjustment.inv_adjustment_line_ids:
        if line.difference_qty != 0:
            impact = line.difference_qty * line.unit_price
            product_impact.append({
                'Product': line.product_id.name,
                'Qty Change': line.difference_qty,
                'Unit Price': line.unit_price,
                'Value Impact': impact,
                'Impact %': (impact / initial_total_value * 100) if initial_total_value else 0
            })
    
    if product_impact:
        impact_df = pd.DataFrame(product_impact)
        print(impact_df.to_string(index=False))
    
    # Check valuation layers if available
    print("\n=== Stock Valuation Layers ===")
    stock_moves = adjustment.inv_adjustment_line_ids.mapped('stock_move_ids')
    if stock_moves:
        layers = env['stock.valuation.layer'].search([
            ('stock_move_id', 'in', stock_moves.ids)
        ])
        
        if layers:
            layer_data = []
            for layer in layers:
                layer_data.append({
                    'Product': layer.product_id.name,
                    'Quantity': layer.quantity,
                    'Unit Cost': layer.unit_cost,
                    'Value': layer.value,
                    'Remaining Qty': layer.remaining_qty,
                    'Remaining Value': layer.remaining_value
                })
            
            layers_df = pd.DataFrame(layer_data)
            print(layers_df.to_string(index=False))
        else:
            print("No valuation layers found")
    
    # Final stock by lot
    print("\n=== Final Stock by Lot (After Adjustment) ===")
    quants = env['stock.quant'].search([
        ('location_id', '=', TEST_CONFIG['location_id']),
        ('quantity', '>', 0)
    ])
    
    final_stock = []
    for quant in quants.sorted('product_id'):
        final_stock.append({
            'Product': quant.product_id.name,
            'Lot': quant.lot_id.name if quant.lot_id else 'No Lot',
            'Quantity': quant.quantity,
            'In Date': quant.in_date.strftime('%Y-%m-%d') if quant.in_date else 'N/A'
        })
    
    if final_stock:
        final_stock_df = pd.DataFrame(final_stock)
        print(final_stock_df.to_string(index=False))
    
    print("\n" + "="*60)
    print("✅ VALUATION ANALYSIS COMPLETE")
    print("="*60)

## 10. Test Summary and Validation

In [ ]:
print("\n" + "#" * 60)
print("TEST EXECUTION SUMMARY")
print("#" * 60)

test_results = {
    'Test Date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'Adjustment Name': adjustment.name,
    'Location': adjustment.location_id.complete_name,
    'Backdate Applied': f"{TEST_CONFIG['backdate_days']} days",
    'Products Tested': len(TEST_CONFIG['products']),
    'Lots Used': len(TEST_CONFIG['lots']),
    'Initial Stock Value': f"${initial_total_value:,.2f}",
    'Final Stock Value': f"${final_total_value:,.2f}",
    'Cost Impact': f"${actual_cost_impact:,.2f}",
    'Stock Moves Created': len(stock_moves),
    'Workflow Completed': adjustment.state == 'done'
}

print("\n📋 Test Results:")
for key, value in test_results.items():
    print(f"  {key}: {value}")

# Validation checks
print("\n✓ Validation Checks:")
validations = [
    ('Backdated refresh applied', adjustment.fetch_type == 'backdated'),
    ('Inventory date updated', adjustment.inventory_date.date() < datetime.now().date()),
    ('Lot lines computed', all(line.adjustment_line_lot_ids for line in adjustment.inv_adjustment_line_ids if line.product_id.tracking == 'lot')),
    ('Stock moves created', len(stock_moves) > 0),
    ('Unit prices stored', all(line.unit_price > 0 for line in adjustment.inv_adjustment_line_ids if line.difference_qty != 0)),
    ('Posted date set', adjustment.posted_date is not None),
    ('Adjustment completed', adjustment.state == 'done')
]

all_passed = True
for check_name, check_result in validations:
    status = "✅ PASS" if check_result else "❌ FAIL"
    print(f"  [{status}] {check_name}")
    if not check_result:
        all_passed = False

print("\n" + "=" * 60)
if all_passed:
    print("🎉 ALL TESTS PASSED SUCCESSFULLY!")
else:
    print("⚠️ SOME TESTS FAILED - Review the results above")
print("=" * 60)

# Export results to JSON for further analysis
import json
with open('test_results.json', 'w') as f:
    json.dump(test_results, f, indent=2)
    print("\n📄 Test results exported to test_results.json")

## 11. Cleanup (Optional)

In [ ]:
# Run this cell only if you want to clean up test data
cleanup = False  # Set to True to enable cleanup

if cleanup:
    with registry.cursor() as cr:
        env = api.Environment(cr, uid, context)
        
        print("Cleaning up test data...")
        
        # Remove test adjustment
        adjustment = env['stock.adjustment.barcode'].browse(adjustment_id)
        if adjustment.exists():
            # Note: May not be able to delete if posted
            if adjustment.state != 'done':
                adjustment.unlink()
                print("✅ Test adjustment removed")
            else:
                print("⚠️ Cannot remove posted adjustment")
        
        # Clean up quants (be careful with this!)
        # test_quants = env['stock.quant'].search([
        #     ('location_id', '=', TEST_CONFIG['location_id']),
        #     ('product_id', 'in', [p['id'] for p in TEST_CONFIG['products']])
        # ])
        # test_quants.unlink()
        # print(f"✅ {len(test_quants)} test quants removed")
        
        cr.commit()
        print("Cleanup completed")
else:
    print("Cleanup skipped (set cleanup=True to enable)")

## Notes for Customization

### To customize this notebook for your specific needs:

1. **Update TEST_CONFIG** in Section 2:
   - Set your actual company_id and location_id
   - Define your actual products with IDs, barcodes, and costs
   - Configure lots for your products
   - Adjust backdate_days and scan_quantities

2. **Modify Initial Stock** in the create_initial_stock function:
   - Set specific quantities for each lot
   - Configure different costs per lot
   - Adjust the in_dates for FIFO testing

3. **Database Connection** in Section 1:
   - Update the Odoo path
   - Set correct config file path
   - Change database name

4. **Add Custom Validations** in Section 10:
   - Add your specific business rules
   - Include custom calculations
   - Export additional data as needed

### Running the Notebook:
- Execute cells sequentially from top to bottom
- Each section is independent after initial setup
- Results are displayed inline with pandas DataFrames
- Final results are exported to JSON for analysis